In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Feb  8 15:45:44 2022

@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
#import pyautogui
from selenium.webdriver.common.action_chains import ActionChains
#import shutil
#import glob
import os
import warnings
#import itertools
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
#import sys
warnings.filterwarnings("ignore")


In [2]:
#open a new window, load the website and logs in

driver = webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')


RL = 'https://home.edweb.net/webinars/'
driver.get(RL)
try:
    driver.maximize_window()
except:
    pass

In [3]:
time.sleep(20)

In [4]:
data = []
prev = ''
while True:
    
    html = driver.find_element_by_class_name('tribe-events-pro-photo').get_attribute('outerHTML')
    time.sleep(5)
    soup = bs(html,'lxml')
    article = soup.find_all('article')
    for i in article:
        date = i.find('time')['datetime']
        title = i.find('h3').text.strip()
        time_range = i.find('div',{'class':'tribe-events-pro-photo__event-datetime tribe-common-b2'}).text.replace('\t','').replace('\n','')
        web=i.find('a')['href']
        data.append(web)
    time.sleep(5)
    driver.find_element_by_class_name('tribe-events-c-nav__next-label').click()
    strUrl = driver.current_url
    time.sleep(2)
    if prev != strUrl:
        prev = strUrl
        #print(strUrl)
    else:
        break


In [6]:
edweb=[]

for za in data:
    driver.get(za)
    ################################
    link=za
    #################################
    try:
        htmln = driver.page_source
        soupn = bs(htmln,'lxml')
        dati= soupn.find('div',{'class':'tribe-events-meta-group tribe-events-meta-group-details'})
        cs=soupn.find('div',{'class':'tribe-events-single-event-description tribe-events-content'})
        alli=soupn.find('div',{'class':'tribe-events-single'})

    except WebDriverException:
        time.sleep(3)
        htmln = driver.page_source
        soupn = bs(htmln,'lxml')
        dati= soupn.find('div',{'class':'tribe-events-meta-group tribe-events-meta-group-details'})
        alli=soupn.find('div',{'class':'tribe-events-single'})
        cs=soupn.find('div',{'class':'tribe-events-single-event-description tribe-events-content'})
    ####################################
    try:
        title=alli.find('h1',{'class':'tribe-events-single-event-title'}).text  
    except:
        title=''
    try:
        dat=dati.find('abbr',{'class':'tribe-events-abbr tribe-events-start-date published dtstart'}).text.strip()
    except:
        dat=''
    try:
        tim=dati.find('div',{'class':'tribe-events-abbr tribe-events-start-time published dtstart'}).text.replace('\t\t','').replace('\n','').split('\t')[0].split('-')
        zone=dati.find('span',{'class':'tribe-events-abbr tribe-events-time-zone published'}).text
    except:
        tim=''
        zone=''
    if dat=="":
        start_date=end_date=''
    else:
        try:
            spl_dt_obj = datetime.strptime(dat, '%A, %B %d, %Y')
            date_= spl_dt_obj.strftime('%Y-%m-%d')
            start_date=end_date=date_
        except:
            start_date=end_date=''
    if (tim=='' or zone==''):
        timer=''
    else:
        timer=[q.strip().split(' ')[0]+q.strip().split(' ')[1].upper() for q in tim]
        timer.append(zone)
    if timer=='':
        time_st=''
    else:
        time_st={'type': 'general',
             'start_time':timer[0],
             'end_time': timer[1],
             'timezone': timer[2],
             'days': 'all'}
    if time_st=='':
        time_=''
    else:
        time_=json.dumps(time_st)
    try:
        event_info=driver.find_element_by_xpath('/html/body/div[8]/div[3]/div/article/div/main/div[2]/div[3]/div[2]/p[5]').text
    except:
        event_info=''
    ticket_list=''
    org_web='https://home.edweb.net/'
    org_name='edWeb'
    mail_='help@edweb.net'
    org_pro=''
    logo=''
    try:
        spons=cs.find('a',{'rel':'noopener'}).text
    except:
        spons=''
    if spons=='':
        sponsor=''
    else:
        sponsor=json.dumps([spons])
    agendalist=''
    type_=''
    category=''
    event_web=link
    Speakerlist=''
    on_off='1'
    city=''
    country=''
    venue=''
    
    def get_google_map_url(location):
        try:
            google_url_for_location="https://www.google.com/search?q="+location+"&oq="+location+"&num=1"
            time.sleep(randint(0,3))
            driver.get(google_url_for_location)
            google_map_url=driver.find_element_by_id('lu_map').click()
            time.sleep(1)
            google_map_url=driver.current_url
            return(google_map_url)
        except Exception as e:
            print(e)
            return("")
    if venue!='':
        googlePlaceUrl=get_google_map_url(venue)
    else:
        googlePlaceUrl=''
    
        
    edweb.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                 org_pro,org_name,org_web,logo,sponsor,agendalist,
                 type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                 Speakerlist,on_off])

In [8]:
edweb_dict=edweb


edweb_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=edweb_dict)
edweb_df.to_csv("edweb.tsv", sep = '\t',index=False)
